In [ ]:
import sys
sys.path.append('../src')
from data_preprocessing import ColoRANDataProcessor, MemoryOptimizedFeatureProcessor, save_processed_data
from federated_learning import model_fn, client_optimizer_fn, server_optimizer_fn, PrivacyBudgetManager
from monitoring import plot_training_history, plot_client_performance, plot_scenario_predictions, plot_training_comparison
from utils import save_model_robust, load_artifacts, predict_efficiency, verify_model_loading
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## 1. Data Preprocessing

In [ ]:
DATASET_PATH = "../data/colosseum-oran-coloran-dataset"

data_processor = ColoRANDataProcessor(DATASET_PATH)
data_processor.download_dataset()
bs_df, ue_df, slice_df = data_processor.load_all_data()

feature_processor = MemoryOptimizedFeatureProcessor(data_processor.slice_configs)
processed_data = feature_processor.process_in_batches(slice_df)

feature_columns = [
    'num_ues', 'slice_id', 'sched_policy_num', 'allocated_rbgs', 'bs_id', 'exp_id',
    'sum_requested_prbs', 'sum_granted_prbs', 'prb_utilization', 'throughput_efficiency',
    'qos_score', 'network_load', 'hour', 'minute', 'day_of_week'
]
available_features = [f for f in feature_columns if f in processed_data.columns]

save_processed_data(processed_data, available_features, 'coloran_processed_features.parquet', 'feature_metadata.json')

## 2. Federated Learning Setup

In [ ]:
INPUT_SHAPE = (len(available_features),)
CLIENT_LEARNING_RATE = 5e-4
SERVER_LEARNING_RATE = 0.01
NUM_ROUNDS = 30
CLIENTS_PER_ROUND = 5
NUM_TOTAL_CLIENTS = 7
DP_L2_NORM_CLIP = 1.0
DP_NOISE_MULTIPLIER = 0.1
DP_BATCH_SIZE = 64
DP_MICROBATCH_SIZE = 1
DP_TARGET_DELTA = 1e-5
DP_TARGET_EPSILON = 10.0
ESTIMATED_CLIENT_SIZE = 10000

iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=lambda: model_fn(INPUT_SHAPE),
    client_optimizer_fn=lambda: client_optimizer_fn(CLIENT_LEARNING_RATE, DP_L2_NORM_CLIP, DP_NOISE_MULTIPLIER, DP_MICROBATCH_SIZE),
    server_optimizer_fn=lambda: server_optimizer_fn(SERVER_LEARNING_RATE)
)

evaluation_process = tff.learning.algorithms.build_fed_eval(lambda: model_fn(INPUT_SHAPE))

## 3. Training

In [ ]:
state = iterative_process.initialize()
history = []
privacy_budget_manager = PrivacyBudgetManager(DP_TARGET_EPSILON, DP_TARGET_DELTA, NUM_ROUNDS, ESTIMATED_CLIENT_SIZE)

for round_num in tqdm(range(NUM_ROUNDS)):
    # This is a simplified training loop. The full implementation is in the original notebook.
    # For demonstration purposes, we'll just simulate the training.
    pass